In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
mobility = pd.read_csv('../data/mobility.csv')

In [ ]:
mobility['County FIPS'] = mobility['County FIPS'].astype('int64')

In [ ]:
##rwjf
rwjf = pd.read_csv('../data/ACS_RWJF.csv').drop(columns=['Median house value'])
house_income = pd.read_csv('../data/median_household_income.csv')
forborn = pd.read_csv('../data/foreign_born.csv')
rwjf = rwjf.merge(house_income,on='FIPS Code',how='inner')
rwjf = rwjf.merge(forborn, on='FIPS Code',how ='inner')

##ccvi
ccvi = pd.read_excel('../data/ccvi.xlsx',engine='openpyxl')

## series
# series_county = pd.read_json('../dataset/completed_series_percent_090921.json', orient='records')
series_county = pd.read_csv('../data/completed_series_percent_013122.csv').drop(columns = ['Unnamed: 0'])
pvi = pd.read_csv('../data/PVI.csv')
pvi = pvi.rename(columns={"casrn":"FIPS Code"}).drop(columns=['row','sid','name','Cases','Deaths'])
series_county = series_county.merge(rwjf, on='FIPS Code', how = 'inner')
series_county = series_county.merge(ccvi, on='FIPS Code', how = 'inner')
series_county = series_county.merge(pvi, on='FIPS Code', how = 'inner')

In [ ]:
static = series_county.drop_duplicates(subset=['FIPS Code'])

In [ ]:
feature_static = [
'Estimated percent of adults ever diagnosed with high blood press',
 'Estimated number of adults ever diagnosed with high cholesterol ',
 'Estimated percent of adults ever diagnosed with a stroke in 2018',
 'Estimated percent of adults reporting to have asthma in 2018.',
 'Estimated percent of adults ever diagnosed with chronic obstruct',
 'Estimated percent of adults reporting to have ever smoked cigare',
 'Annual cancer incidence per 100,000 persons, as measured in 2011',
 'Rate of persons living with an HIV diagnosis per 100,000 people ',
 'Estimated percent of adults reporting to be overweight (a body m',
 'Estimated percent of adults ever diagnosed with diabetes in 2018',
 'Number of deaths due to influenza and pneumonia per 100,000 people',
 'Below Poverty',
 'Unemployed',
 'Income',
 'No High School Diploma',
 'Aged 65 or Older',
 'Aged 17 or Younger',
 'Older than Age 5 with a Disability',
 'Single-Parent Households',
 'Minority',
 'Speaks English “Less than Well”',
 'Multi-Unit Structures',
 'Mobile Homes',
 'Crowding',
 'No Vehicle',
 'Group Quarters',
 'E_UNINSUR',
 'Number of intensive care unit (ICU) beds in 2019.',
 'Rate of hospital beds per 1,000 people in 2016.',
 'Aggregate cost of medical care in 2017.',
 'Rate of emergency room visits per 1,000 people in 2016.',
 'Percentage of population employed in high-risk industry',
 'Long-term care (nursing homes, assisted living, and care homes) residents per 100,000',
 'Prisons population per 100000',
 'Receipt of government financial assistance ',
 'sex ratio',
'Gini Index of Income Inequality',
'Estimated percent of all people who were foreign born as of 2015',
'Traffic','AirPollution'
]

In [ ]:
existed_mobility = mobility.merge(static, left_on = 'County FIPS', right_on ='FIPS Code' ).drop_duplicates(subset=['FIPS Code'])

In [ ]:
picked_counties = existed_mobility[feature_static].fillna(0)

In [ ]:
kmeans = KMeans(n_clusters=180).fit(picked_counties)

In [ ]:
results = kmeans.predict(picked_counties)

In [ ]:
existed_mobility['clusters'] = results
existed_mobility = existed_mobility.fillna(0)

In [ ]:
med = existed_mobility.groupby(by=["clusters"]).median()

In [ ]:
clustered_common = existed_mobility[['clusters', 'County FIPS']].rename(columns={'County FIPS':'FIPS Code'})
clustered_common

In [ ]:
# histogram of clusters
ax = existed_mobility['clusters'].plot.hist(bins=180, alpha=0.5)

In [ ]:
df_all = static.merge(mobility, left_on = 'FIPS Code', right_on = 'County FIPS',how='left', indicator=True)

In [ ]:
no_mobility = df_all[df_all['_merge']=='left_only'][feature_static + ['FIPS Code']].fillna(0)

In [ ]:
result = kmeans.predict(no_mobility[feature_static])

In [ ]:
no_mobility['clusters'] = result

In [ ]:
no_mobility_index = no_mobility[['FIPS Code', 'clusters']].reset_index(drop=True)

In [ ]:
no_mobility_index

In [ ]:
whole_cluster = clustered_common.append(no_mobility_index)

In [ ]:
whole_cluster.to_csv('./whole_cluster.csv', index= False)

In [ ]:
ax = no_mobility['clusters'].plot.hist(bins=75, alpha=0.5)

In [ ]:
No_mobility_whole = series_county.merge(no_mobility_index, on = 'FIPS Code', how = 'inner')
No_mobility_whole['date'] = pd.to_datetime(No_mobility_whole['date'])

In [ ]:
No_mobility_whole

In [ ]:
#import dot_mobility
dot_data = pd.read_csv('../data/data_dot.csv').drop(columns = ['Level','State FIPS',
                                                              'STATEAB', 'County Name','Row ID','Week','Month'])
dot_data['County FIPS'] = dot_data['County FIPS'].apply('int64')
dot_data = dot_data.rename(columns={'Number of Trips <1':'Number of Trips 1', 'Number of Trips >=500':
                                                     'Number of Trips 500', 'County FIPS': 'FIPS Code','Date': 'date'})
dot_data['date'] = pd.to_datetime(dot_data['date'])

In [ ]:
series_county['date'] = pd.to_datetime(series_county['date'])
mobility['date'] = pd.to_datetime(mobility['date'])
mobility = mobility.drop(columns = ['state','county','timestamp','Level','Date','State FIPS','STATEAB','County Name'])
mobility = mobility.rename(columns={'Number of Trips <1':'Number of Trips 1', 'Number of Trips >=500':
                                                     'Number of Trips 500', 'County FIPS': 'FIPS Code'})

In [ ]:
No_mobility_whole = No_mobility_whole.merge(dot_data, on =['FIPS Code', 'date']).reset_index(drop = True)

In [ ]:
With_mobility_whole = series_county.merge(mobility, on = ['FIPS Code','date'], how = 'inner')

In [ ]:
With_mobility_whole_new = With_mobility_whole.merge(clustered_common, on = 'FIPS Code')

In [ ]:
imputation = With_mobility_whole_new[['date','clusters','retail_recreation',
 'grocery_pharmacy',
 'parks',
 'transit_stations',
 'workplaces',
 'residential']]

In [ ]:
temp = imputation.fillna(0).groupby(['clusters','date']).median()

In [ ]:
temp['index'] = temp.index.values

In [ ]:
temp['clusters'] = temp['index'].apply(lambda x: x[0])
temp['date'] = temp['index'].apply(lambda x: x[1])
temp['date'] = pd.to_datetime(temp['date'])

In [ ]:
imputation = temp.reset_index(drop = True).drop(columns = ['index'])

In [ ]:
imputation

In [ ]:
data_no_mobility = No_mobility_whole.merge(imputation, on = ['clusters', 'date'])
var_change = ['Population Staying at Home',
 'Population Not Staying at Home',
 'Number of Trips',
 'Number of Trips 1',
 'Number of Trips 1-3',
 'Number of Trips 3-5',
 'Number of Trips 5-10',
 'Number of Trips 10-25',
 'Number of Trips 25-50',
 'Number of Trips 50-100',
 'Number of Trips 100-250',
 'Number of Trips 250-500',
 'Number of Trips 500']
for i in var_change:
    data_no_mobility[i] = data_no_mobility[i] / data_no_mobility['Population']

In [ ]:
data_no_mobility.to_csv('../data/hold-out-set.csv', index = False)